<a href="https://colab.research.google.com/github/singhsourav0/Deep-Learning-Odyssey/blob/main/LSTM_Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c fake-news


 88% 41.0M/46.5M [00:00<00:00, 132MB/s]
100% 46.5M/46.5M [00:00<00:00, 128MB/s]


In [6]:
!unzip fake-news.zip

Archive:  fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("train.csv")

In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
df = df.dropna()

In [11]:
x = df.drop('label', axis=1)

In [12]:
y =df['label']

In [13]:
x.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf


In [16]:
tf.__version__

'2.15.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [18]:
voc_size = 5000

One Hot Encoding

In [19]:
message = x.copy()

In [20]:
message['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
message.reset_index(inplace= True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[1177, 332, 3386, 536, 3304, 1511, 1978, 1824, 4066, 3762],
 [849, 2105, 3861, 102, 1738, 2413, 4302],
 [2402, 3991, 2401, 3721],
 [4022, 515, 4885, 3777, 3005, 3681],
 [76, 1738, 1043, 3357, 4725, 4479, 1738, 3502, 367, 2223],
 [282,
  2158,
  4545,
  3850,
  4256,
  3263,
  4307,
  4836,
  1655,
  1106,
  1695,
  1484,
  2709,
  2738,
  4302],
 [3816, 3017, 2715, 770, 4230, 3556, 2405, 2540, 1200, 3636, 3677],
 [2088, 764, 466, 2608, 3460, 4519, 3263, 3848, 1200, 3636, 3677],
 [3335, 4397, 390, 4878, 349, 876, 2464, 1270, 3263, 269],
 [1531, 3520, 4406, 1786, 3769, 2476, 1043, 1683],
 [1976, 931, 4423, 1671, 3573, 3547, 1131, 4188, 3150, 975, 4458],
 [3777, 4446, 3304, 876, 3263, 3460],
 [527, 215, 3553, 4361, 571, 1499, 4764, 4225, 1131],
 [357, 564, 3088, 383, 565, 4387, 3304, 1200, 3636, 3677],
 [2895, 187, 2983, 1089, 1339, 1200, 3636, 3677],
 [3482, 4147, 4885, 1644, 2069, 4888, 3615, 958, 4793, 2940],
 [1618, 4135, 2105],
 [891, 2367, 3933, 1899, 3263, 2278, 3411, 4302],
 [254

Embedding Representation

In [27]:
sent_length =20
embedded_docs =pad_sequences(onehot_repr, padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1824 4066 3762]
 [   0    0    0 ... 1738 2413 4302]
 [   0    0    0 ... 3991 2401 3721]
 ...
 [   0    0    0 ... 1200 3636 3677]
 [   0    0    0 ... 1531 1967  563]
 [   0    0    0 ... 4243 4437  344]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1177,
        332, 3386,  536, 3304, 1511, 1978, 1824, 4066, 3762], dtype=int32)

In [29]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:

len(embedded_docs),y.shape

(18285, (18285,))

In [31]:
import numpy as np
x_final =np.array(embedded_docs)
y_final =np.array(y)

In [32]:
x_final.shape, y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [34]:
### Finally Training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 42ms/step - loss: 0.3414 - accuracy: 0.8409 - val_loss: 0.1973 - val_accuracy: 0.9158
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1409 - accuracy: 0.9456 - val_loss: 0.1972 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0984 - accuracy: 0.9627 - val_loss: 0.2149 - val_accuracy: 0.9198
Epoch 4/10
192/192 [==============================] - 6s 29ms/step - loss: 0.0692 - accuracy: 0.9762 - val_loss: 0.2296 - val_accuracy: 0.9167
Epoch 5/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0448 - accuracy: 0.9853 - val_loss: 0.2688 - val_accuracy: 0.9145
Epoch 6/10
192/192 [==============================] - 6s 30ms/step - loss: 0.0263 - accuracy: 0.9921 - val_loss: 0.3772 - val_accuracy: 0.9117
Epoch 7/10
192/192 [==============================] - 7s 39ms/step - loss: 0.0162 - accuracy: 0.9949 - val_loss: 0.4233 - val_accuracy: 0.909

In [35]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [36]:
y_pred=model.predict(x_test)

189/189 [==============================] - 2s 7ms/step


In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
y_test = y_test.astype(np.int64)
y_pred = y_pred.astype(np.int64)

# Recalculate the confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print(confusion_matrix)

[[3419    0]
 [2616    0]]


In [39]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917